In [1]:
import pandas as pd
import numpy as np
import requests
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
plt.style.use("seaborn")

/home/pnietoramos/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Defining Features and Classifier

In [2]:
df_multi = pd.read_csv("Fire_multi.csv", index_col=False)

In [3]:
df_multi.head(1)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Soil_Type
0,2596,51,3,258,0,510,221,232,148,0,5,29


In [4]:
X = df_multi.drop("Horizontal_Distance_To_Fire_Points",axis=1)
y = df_multi["Horizontal_Distance_To_Fire_Points"]

In [5]:
#Splitting the set (test = 0.3) in a stratified manner
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30,stratify=y, random_state=4444)

In [7]:
#scaling the features
sca = StandardScaler()
X_train_sca = sca.fit_transform(X_train)
X_test_sca = sca.transform(X_test);

In [8]:
def grid_estimator(estimator, param_grid, X_train, y_train, X_test, y_test, cv=5):
    grid = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=cv, n_jobs=1)
    grid.fit(X_train, y_train)
    print("Accuracy training: " + str(grid.best_score_))
    y_pred = grid.predict(X_test)
    print("Accuracy test: " + str(accuracy_score(y_pred,y_test)))
    print(grid.best_estimator_)


### Trying SVC with default values (TAKES TOO LONG!!!)

In [ ]:
param_grid = {}
grid_estimator(SVC(), param_grid, X_train_sca, y_train, X_test_sca, y_test)

In [ ]:
conf_trees(y_test, y_pred)

In [ ]:
print("Recall for cover:")
for i in range(3):
    print("Type " + str(i)+ ":" + str(100 * confusion_matrix(y_test, y_pred)[i,i]/len(y_test[y_test==(i)])))

In [ ]:
print("Precision for cover:")
for i in range(3):
    print("Type " + str(i)+ ":" + str(100 * confusion_matrix(y_test, y_pred)[i,i]/len(y_pred[y_pred==(i)])))

### SGDClassifier

In [ ]:
param_grid = {"penalty":["l1","l2"], "alpha":[0.01,0.1, 1, 10, 100], "max_iter": [1000,10000]}
grid_estimator(SGDClassifier(), param_grid, X_train_sca, y_train, X_test_sca, y_test)

### Selecting features with importance >0.07  (Random Forest)

In [ ]:
col_in = ["Elevation","Horizontal_Distance_To_Roadways", "Soil_Type",
          "Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology"]
X_in = X[col_in]
X_train_in = X_train[col_in]
X_test_in = X_test[col_in]

In [ ]:
param_grid = {"C": [0.1,1, 10, 100, 1000], "gamma": [1/15, 1/10, 1/5, 1, 10]}
grid_estimator(SVC(), param_grid, X_train_in, y_train, X_test_in, y_test)